# Automated Machine Learning の使用

モデルのトレーニングに使用できる機械学習アルゴリズムにはさまざまな種類があり、特定のデータや予測要件に最も効果的なアルゴリズムを特定するのは容易ではない場合があります。さらに、正規化、欠落したフィーチャの代入などの手法を使用して、トレーニング データを前処理することによってモデルの予測パフォーマンスに大きな影響を与えることができます。要件に*最適*なモデルを見つけるには、多くのアルゴリズムと前処理変換の組み合わせを試す必要があります。これには多くの時間とコンピューティング リソースがかかります。

Azure Machine Learning により、さまざまなアルゴリズムと前処理オプションを使用して、トレーニング済みモデルの比較を自動化できます。[Azure Machine Learning Studio](https://ml/azure.com) または SDK のビジュアル インターフェイスを使用してこの機能を活用できます。SDKを使用すると、自動化された Machine Learning 実験の設定をより細かく制御できますが、視覚的なインターフェイスは使いやすくなっています。このラボでは、SDK を使用した機自動機械学習について説明します。

## ワークスペースに接続する

まず、Azure ML SDK を使用してワークスペースに接続する必要があります。

> **注**: 前回の演習を完了してから Azure サブスクリプションとの認証済みセッションの有効期限が切れている場合は、再認証を求めるメッセージが表示されます。

In [ ]:
import azureml.core
from azureml.core import Workspace

# 保存した構成ファイルからワークスペースを読み込む
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

## 自動機械学習用データを準備する

自動機械学習用トレーニング スクリプトを作成する必要はありませんが、トレーニング データを読み込む必要があります。この場合、糖尿病患者の詳細を含むデータセットを作成し (前の実習で行ったのと同じように)、これをトレーニング用とモデル検証用の 2 つのデータセットに分割します。

In [ ]:
from azureml.core import Dataset

default_ds = ws.get_default_datastore()

if 'diabetes dataset' not in ws.datasets:
    default_ds.upload_files(files=['./data/diabetes.csv', './data/diabetes2.csv'], # 糖尿病 CSV ファイルを /data にアップロードする
                        target_path='diabetes-data/', # データストアのフォルダー パスに入れる
                        overwrite=True, # 同じ名前の既存のファイルを置き換える
                        show_progress=True)

    #表形式のデータセットを作成して登録する (前のラボが完了していない場合)
    tab_data_set = Dataset.Tabular.from_delimited_files(path=(default_ds, 'diabetes-data/*.csv'))

    # 表形式データセットを登録する
    try:
        tab_data_set = tab_data_set.register(workspace=ws, 
                                name='diabetes dataset',
                                description='diabetes data',
                                tags = {'format':'CSV'},
                                create_new_version=True)
        print('Dataset registered.')
    except Exception as ex:
        print(ex)
else:
    print('Dataset already registered.')


# トレーニングと検証のサブセットにデータセットを分割する
diabetes_ds = ws.datasets.get("diabetes dataset")
train_ds, test_ds = diabetes_ds.random_split(percentage=0.7, seed=123)
print("Data ready!")

## Automated Machine Learning を構成する

これで、自動 Machine Learning 実験を構成する準備が整いました。これを行うには、実験環境に必要なパッケージを含む実行構成、試行する組み合わせの数、モデルの評価時に使用するメトリックなどを指定する構成設定のセットが必要です。

> **注**: このラボでは、時間を節約するために、最大 6 回の反復でローカル コンピューティングで、自動 Machine Learning 実験を実行します。これにより、各反復 (子の実行) は、並列的ではなく、連続的に実行されます。このため、実験は 6 回に制限され、所要時間を短縮します。実際には、コンピューティング クラスターを使用して、さらに多くの反復を試行する必要があります。

In [ ]:
from azureml.train.automl import AutoMLConfig

automl_config = AutoMLConfig(name='Automated ML Experiment',
                             task='classification',
                             compute_target='local',
                             enable_local_managed=True,
                             training_data = train_ds,
                             validation_data = test_ds,
                             label_column_name='Diabetic',
                             iterations=6,
                             primary_metric = 'AUC_weighted',
                             max_concurrent_iterations=4,
                             featurization='auto'
                             )

print("Ready for Auto ML run.")

## 自動 Machine Learning 実験を実行する

準備完了です。自動 Machine Learning 実験を実行しましょう。

In [ ]:
from azureml.core.experiment import Experiment
from azureml.widgets import RunDetails

print('Submitting Auto ML experiment...')
automl_experiment = Experiment(ws, 'diabetes_automl')
automl_run = automl_experiment.submit(automl_config)
RunDetails(automl_run).show()
automl_run.wait_for_completion(show_output=True)

## 最もパフォーマンスの高いモデルを決定する

実験が完了したら、ウィジェットで出力を表示し、最適な結果を生成した実行をクリックして詳細を表示します。
次に、Azure Portal で実験の詳細を表示し、実験の詳細を表示してから、最良の結果を得た個々の実行の詳細を表示するリンクをクリックします。ここでは、生成されたモデルのパフォーマンスに関する情報が多数あります。

最高の実行とそれが生成したモデルを取得しましょう。

In [ ]:
best_run, fitted_model = automl_run.get_output()
print(best_run)
print(fitted_model)
best_run_metrics = best_run.get_metrics()
for metric_name in best_run_metrics:
    metric = best_run_metrics[metric_name]
    print(metric_name, metric)

自動 Machine Learningには、データの前処理を試すオプションが含まれていますが、これは [Scikit-Learn 変換パイプライン](https://scikit-learn.org/stable/modules/compose.html#combining-estimators)を使用して行われます (Azure Machine Learning パイプラインと混同しないでください)。これらは、推論する前にデータを変換するステップを含むモデルを生成します。次のようなモデルのステップを表示できます。

In [ ]:
for step in fitted_model.named_steps:
    print(step)

最後に、最もパフォーマンスの高いモデルを見つけたので、それを登録できます。

In [ ]:
from azureml.core import Model

# モデルを登録する
best_run.register_model(model_path='outputs/model.pkl', model_name='diabetes_model_automl',
                        tags={'Training context':'Auto ML'},
                        properties={'AUC': best_run_metrics['AUC_weighted'], 'Accuracy': best_run_metrics['accuracy']})

# 登録済みモデルを一覧表示する
for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

> **詳細情報**: 自動 Machine Learning の詳細については、[Azure ML のドキュメント](https://docs.microsoft.com/azure/machine-learning/how-to-configure-auto-train)を参照してください。